In [ ]:
from IPython.display import clear_output
!pip install python-dotenv
clear_output()

In [ ]:
import pandas as pd 
import numpy as np
import os 
import datetime
from tqdm.notebook import tqdm
import dotenv

import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import plotly.express as px

In [ ]:
# local or gdrive
path_source = 'gdrive'

if path_source == 'gdrive':
  from google.colab import drive
  drive.mount('/content/gdrive')
  data_path = '/content/gdrive/MyDrive/Classes/W210_capstone/W210 Capstone/Data/census'
  wind_path = '/content/gdrive/MyDrive/Classes/W210_capstone/W210 Capstone/Data/wind'
  env_path = '/content/gdrive/MyDrive/.env'
  
elif path_source == 'local':
  data_path = '/Users/tj/trevorj@berkeley.edu - Google Drive/My Drive/Classes/W210_capstone/W210 Capstone/Data/census'
  env_path = '/content/gdrive/MyDrive/.env'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from dotenv import dotenv_values

In [ ]:
api_keys = dotenv_values(env_path)

In [ ]:
import json
import urllib

In [ ]:
url = "https://maps.googleapis.com/maps/api/elevation/json" 
lat = '37.800860' 
lon = '-122.436222'
apikey = api_keys['google_maps']

request = urllib.urlopen(url+"?locations="+str(lat)+","+str(lon)+"&key="+apikey)


AttributeError: ignored

In [ ]:


def elevation(lat, lng):
    apikey = "USE YOUR OWN KEY !!!"
    url = "https://maps.googleapis.com/maps/api/elevation/json"
    request = urllib.urlopen(url+"?locations="+str(lat)+","+str(lng)+"&key="+apikey)
    try:
        results = json.load(request).get('results')
        if 0 < len(results):
            elevation = results[0].get('elevation')
            # ELEVATION
            return elevation
        else:
            print 'HTTP GET Request failed.'
    except ValueError, e:
        print 'JSON decode failed: '+str(request)

# Attempt v2

In [ ]:
import requests
import urllib

In [ ]:
# USGS Elevation Point Query Service
url = r'https://nationalmap.gov/epqs/pqs.php?'

# coordinates with known elevation 
lat = [48.633, 48.733, 45.1947, 45.1962]
lon = [-93.9667, -94.6167, -93.3257, -93.2755]

# create data frame
df = pd.DataFrame({
    'lat': lat,
    'lon': lon
})

def elevation_function(df, lat_column, lon_column):
    """Query service using lat, lon. add the elevation values as a new column."""
    elevations = []
    for lat, lon in zip(df[lat_column], df[lon_column]):

        # define rest query params
        params = {
            'output': 'json',
            'x': lon,
            'y': lat,
            'units': 'Meters'
        }

        # format query string and return query value
        result = requests.get((url + urllib.parse.urlencode(params)))
        elevations.append(result.json()['USGS_Elevation_Point_Query_Service']['Elevation_Query']['Elevation'])

    df['elev_meters'] = elevations

elevation_function(df, 'lat', 'lon')
df.head()

,lat,lon,elev_meters
0,48.6330,-93.9667,341.14
1,48.7330,-94.6167,328.80
2,45.1947,-93.3257,262.68
3,45.1962,-93.2755,272.64


In [ ]:
df_wind = pd.read_parquet(wind_path)

Time durations for running the script on the wind data:
- 10 obs: 3 sec
- 100 obs: 30.7 sec 
- 1000 obs: 293 sec (4:53 min)

So it takes .3 sec for every obs. So for 449075 obs it'll take ~37.4 hrs. 

In [ ]:
%%time
df_wind_small = df_wind.head(10)
elevation_function(df_wind_small, 'lat', 'lon')

CPU times: user 152 ms, sys: 5.38 ms, total: 157 ms
Wall time: 2.95 s


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
%%time
df_wind_small = df_wind.head(100)
elevation_function(df_wind_small, 'lat', 'lon')
df_wind_small

CPU times: user 1.49 s, sys: 129 ms, total: 1.62 s
Wall time: 30.7 s


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,lat,lon,ZCTA10,u,v,wdir,wspd,year_month,elev_meters
0,37.465,-117.936,89010,0.815439,0.606772,36.653091,1.016421,199801,1635.34
1,35.396,-116.322,89019,0.282826,0.555248,63.007133,0.623130,199801,744.60
2,36.161,-116.139,89060,-0.143215,0.582321,103.816978,0.599673,199801,773.34
3,35.957,-115.897,89061,0.381607,0.403891,46.624981,0.555654,199801,798.31
4,39.520,-120.032,89439,0.947113,1.138490,50.242836,1.480940,199801,1743.30
...,...,...,...,...,...,...,...,...,...
95,33.977,-118.472,90291,0.629467,-0.685340,312.566650,0.930548,199801,0.00
96,33.959,-118.464,90292,0.629467,-0.685340,312.566650,0.930548,199801,0.00
97,33.943,-118.452,90293,0.629467,-0.685340,312.566650,0.930548,199801,0.00
98,33.945,-118.369,90301,0.281756,-0.200910,324.508789,0.346051,199801,30.87


In [ ]:
%%time
df_wind_small = df_wind.head(1000)
elevation_function(df_wind_small, 'lat', 'lon')
df_wind_small

CPU times: user 14.6 s, sys: 936 ms, total: 15.6 s
Wall time: 4min 53s


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,lat,lon,ZCTA10,u,v,wdir,wspd,year_month,elev_meters
0,37.465,-117.936,89010,0.815439,0.606772,36.653091,1.016421,199801,1635.34
1,35.396,-116.322,89019,0.282826,0.555248,63.007133,0.623130,199801,744.6
2,36.161,-116.139,89060,-0.143215,0.582321,103.816978,0.599673,199801,773.34
3,35.957,-115.897,89061,0.381607,0.403891,46.624981,0.555654,199801,798.31
4,39.520,-120.032,89439,0.947113,1.138490,50.242836,1.480940,199801,1743.3
...,...,...,...,...,...,...,...,...,...
995,38.537,-122.474,94576,0.153312,0.797154,79.113541,0.811763,199801,96.85
996,37.668,-122.162,94577,0.396103,1.290302,72.934319,1.349732,199801,1.57
997,37.689,-122.137,94578,0.396103,1.290302,72.934319,1.349732,199801,8.04
998,37.668,-122.161,94579,0.396103,1.290302,72.934319,1.349732,199801,3.4
